# Structured Output: 구조화된 출력

이 노트북에서는 **LLM의 출력을 특정 구조로 강제**하는 방법을 배웁니다.

## Structured Output이란?

```
┌────────────────────────────────────────────────────────────────────┐
│                    Structured Output의 개념                        │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   일반 LLM 출력:                                                   │
│   "고양이 농담이요? 음... 고양이가 컴퓨터를 좋아하는 이유는..."    │
│   → 자유 형식 텍스트 (파싱하기 어려움)                            │
│                                                                    │
│   Structured Output:                                               │
│   {                                                                │
│     "setup": "고양이가 컴퓨터를 좋아하는 이유는?",               │
│     "punchline": "마우스가 있으니까!"                            │
│   }                                                                │
│   → 정해진 구조로 출력 (파싱 용이, 타입 안전)                     │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## 왜 필요할까요?

```
┌────────────────────────────────────────────────────────────────────┐
│                    Structured Output 사용 사례                     │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   • API 응답 생성: 특정 JSON 형식 필요                            │
│   • 데이터 추출: 이름, 날짜, 금액 등 구조화된 정보 추출           │
│   • 분류 작업: 정해진 카테고리 중 하나 선택                       │
│   • 에이전트 결정: 다음 행동 선택 (Supervisor 패턴)               │
│   • 폼 데이터 생성: 사용자 입력 검증                              │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
!pip install -q langchain langchain-ollama

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. Pydantic 스키마 정의

출력 구조를 Pydantic 모델로 정의합니다.

In [ ]:
from pydantic import BaseModel, Field

class Joke(BaseModel):
    """
    농담 구조
    
    setup: 농담의 설정 ("왜 고양이가...")
    punchline: 농담의 포인트 ("마우스가 있으니까!")
    """
    setup: str = Field(description='농담의 설정')
    punchline: str = Field(description='농담의 포인트')

print("✅ Joke 스키마 정의 완료")
print("   - setup: 농담의 설정")
print("   - punchline: 농담의 포인트")

# 3. LLM에 Structured Output 적용

In [ ]:
from langchain_ollama import ChatOllama

# 기본 모델
base_model = ChatOllama(model='llama3.2', temperature=0)

# Structured Output 적용
model = base_model.with_structured_output(Joke)

print("✅ Structured Output LLM 설정 완료")
print("   항상 Joke 형태로 응답함")

# 4. 실행 및 결과 확인

In [ ]:
# 농담 생성
result = model.invoke('고양이에 대한 농담을 만들어 주세요.')

print("=== Structured Output 결과 ===")
print(f"타입: {type(result)}")
print(f"\n결과: {result}")
print(f"\n설정: {result.setup}")
print(f"포인트: {result.punchline}")

# 5. 다양한 스키마 예시

In [ ]:
from typing import Literal, Optional
from pydantic import BaseModel, Field

# 예시 1: 감정 분석
class SentimentAnalysis(BaseModel):
    sentiment: Literal['positive', 'negative', 'neutral'] = Field(
        description='텍스트의 감정'
    )
    confidence: float = Field(
        description='신뢰도 (0.0 ~ 1.0)',
        ge=0.0,
        le=1.0
    )
    reason: str = Field(description='판단 이유')

# 예시 2: 개체명 인식
class Entity(BaseModel):
    name: str = Field(description='개체 이름')
    type: Literal['person', 'organization', 'location', 'date', 'other'] = Field(
        description='개체 유형'
    )

class EntityExtraction(BaseModel):
    entities: list[Entity] = Field(description='추출된 개체 목록')

# 예시 3: 작업 분류
class TaskClassification(BaseModel):
    category: Literal['question', 'request', 'complaint', 'feedback'] = Field(
        description='작업 카테고리'
    )
    priority: Literal['low', 'medium', 'high', 'urgent'] = Field(
        description='우선순위'
    )
    summary: str = Field(description='요약')

print("✅ 다양한 스키마 정의 완료")

In [ ]:
# 감정 분석 테스트
sentiment_model = base_model.with_structured_output(SentimentAnalysis)

text = "이 제품 정말 최고예요! 배송도 빠르고 품질도 좋습니다."
result = sentiment_model.invoke(f"다음 텍스트의 감정을 분석해주세요: {text}")

print("=== 감정 분석 결과 ===")
print(f"텍스트: {text}")
print(f"\n감정: {result.sentiment}")
print(f"신뢰도: {result.confidence}")
print(f"이유: {result.reason}")

In [ ]:
# 개체명 인식 테스트
entity_model = base_model.with_structured_output(EntityExtraction)

text = "삼성전자의 이재용 회장이 2024년 1월 서울에서 기자회견을 열었습니다."
result = entity_model.invoke(f"다음 텍스트에서 개체를 추출해주세요: {text}")

print("=== 개체명 인식 결과 ===")
print(f"텍스트: {text}")
print(f"\n추출된 개체:")
for entity in result.entities:
    print(f"  - {entity.name} ({entity.type})")

---

## 정리: Structured Output

### 핵심 코드

```python
from pydantic import BaseModel, Field

# 1. 스키마 정의
class MyOutput(BaseModel):
    field1: str = Field(description='설명')
    field2: int = Field(description='설명')

# 2. LLM에 적용
model = base_model.with_structured_output(MyOutput)

# 3. 실행 (결과가 MyOutput 타입)
result = model.invoke('프롬프트')
print(result.field1)  # 타입 안전
```

### Field 옵션

| 옵션 | 설명 | 예시 |
|------|------|------|
| **description** | 필드 설명 | `Field(description='이름')` |
| **default** | 기본값 | `Field(default='기본')` |
| **ge/le** | 숫자 범위 | `Field(ge=0, le=100)` |
| **min_length** | 문자열 최소 길이 | `Field(min_length=1)` |

### Literal로 선택지 제한

```python
from typing import Literal

class Decision(BaseModel):
    choice: Literal['option1', 'option2', 'option3']  # 이 중 하나만 가능
```

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
model = ChatOpenAI(model='gpt-4o-mini', temperature=0)
model = model.with_structured_output(Joke)

# 변경
model = ChatOllama(model='llama3.2', temperature=0)
model = model.with_structured_output(Joke)
```

## 다음 단계

**Streaming**을 사용하여 실시간으로 출력을 받는 방법을 배웁니다. (03번 노트북)